In [ ]:
from PIL import Image
import cv2
import numpy as np
from pdf2image import convert_from_path

In [ ]:
#Detecting paper

# Path to the PDF file and the photo
pdf_path = 'OPH_project\Adk\Passport v18.pdf' # Passport pdf
photo_path = 'path_to_your_photo.jpg'  # path to photo

# Convert the first page of the PDF to an image
pages = convert_from_path(pdf_path, first_page=0, last_page=1)
pdf_image = np.array(pages[0])

# Resize the PDF image for matching purposes
pdf_gray = cv2.cvtColor(pdf_image, cv2.COLOR_BGR2GRAY)
pdf_gray = cv2.resize(pdf_gray, (0, 0), fx=0.5, fy=0.5)

# Load the photo and convert to grayscale
photo = cv2.imread(photo_path)
photo_gray = cv2.cvtColor(photo, cv2.COLOR_BGR2GRAY)

# Apply GaussianBlur to reduce noise and improve edge detection
blurred = cv2.GaussianBlur(photo_gray, (5, 5), 0)

# Perform edge detection
edges = cv2.Canny(blurred, 50, 150)

# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Sort contours by area and iterate through them to find the best match
contours = sorted(contours, key=cv2.contourArea, reverse=True)
best_match = None
best_match_value = float('inf')
best_rect = None

for contour in contours:
    x, y, w, h = cv2.boundingRect(contour)
    roi = photo_gray[y:y+h, x:x+w]
    roi_resized = cv2.resize(roi, (pdf_gray.shape[1], pdf_gray.shape[0]))
    match_value = cv2.matchTemplate(roi_resized, pdf_gray, cv2.TM_SQDIFF_NORMED)[0][0]
    if match_value < best_match_value:
        best_match_value = match_value
        best_match = contour
        best_rect = (x, y, w, h)

# Draw the best match contour on the original photo
if best_match is not None and best_rect is not None:
    x, y, w, h = best_rect
    cv2.drawContours(photo, [best_match], -1, (0, 255, 0), 2)
    
    # Crop the detected part
    cropped_image = photo[y:y+h, x:x+w]
    
    # Save the cropped image as JPG
    cv2.imwrite('mainPicSave/detected_paper.jpg', cropped_image)
    print("Cropped image saved as 'detected_paper.jpg'")

# Display the original photo with the detected paper edges
cv2.imshow('Detected Paper', photo)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
#Function resize image for YOLO

In [ ]:
#crop upper part

image_path = 'path/to/your/image.jpg'  # Change this to the path of your image
img = Image.open(image_path)

width_ratios = [10, 22, 34, 46, 59, 71]

width, height = img.size
crop_widths = [int(width * (ratio / 71)) for ratio in width_ratios]
previous_width=0
for i, crop_width in enumerate(crop_widths):
    crop_box = (previous_width, 0, crop_width, height*(28/53))
    cropped_img = img.crop(crop_box)
    cropped_image_path = f'path/to/save/cropped_image_section_{i + 1}.jpg'  # Change this to your desired save path
    cropped_img.save(cropped_image_path)
    #print(f"Cropped image section {i + 1} saved at {cropped_image_path}")
    
    previous_width = crop_width

#print("All sections have been cropped and saved.")


In [ ]:
#crop lower part

crop_widths = [int(width * (ratio / 71)) for ratio in width_ratios]
previous_width=0
for i, crop_width in enumerate(crop_widths):
    crop_box = (previous_width, height*(28/53), crop_width, height)
    cropped_img = img.crop(crop_box)
    cropped_image_path = f'path/to/save/cropped_image_section_{i + 1 + 6}.jpg'  # Change this to your desired save path
    cropped_img.save(cropped_image_path)
    #print(f"Cropped image section {i + 1} saved at {cropped_image_path}")
    
    previous_width = crop_width